PUT vs POST/GET:

GET: Read data only.

POST: Create new data.

PUT: Update existing data completely (idempotent).

DELETE: Remove data.

******************************************** 

POST is not idempotent (sending twice creates duplicates).

PUT is idempotent (sending twice has same effect).

Makes API behavior predictable and follows REST standards.

*************************************************************** 
Usefulness in FastAPI for AI projects:

Updating AI model configs, datasets, or training records.

Managing experiment metadata.



POST is not idempotent (sending twice creates duplicates).

In [27]:
import os , sys
# ensure parent folder is in Python path
sys.path.append("/home/ujjwal/learning/fastapi_learn")

# change to parent folder of 'basic'
os.chdir("/home/ujjwal/learning/fastapi_learn")

!pwd


/home/ujjwal/learning/fastapi_learn


In [28]:
%%writefile basic/put.py


from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

reviews_table = {
    1: {"_id": 1, "movie": "Inception", "num_stars": 5, "text": "Great!"},
    2: {"_id": 2, "movie": "Matrix", "num_stars": 4, "text": "Awesome!"}
}

class Dbreview(BaseModel):
    movie: str
    num_stars: int
    text: str
    review_id: int

@app.get("/")
def home_page():
    return "Welcome to the home page"


@app.put("/reviews/")
def update_review(review: Dbreview):
    reviews_table[review.review_id] = {
        "_id": review.review_id,
        "movie": review.movie,
        "num_stars": review.num_stars,
        "text": review.text
    }
    return {"status": "updated", "review": reviews_table[review.review_id]}

@app.get("/reviews")
def get_all_reviews():
    return reviews_table


Overwriting basic/put.py


In [ ]:
import os, subprocess, sys
import os
os.system("fuser -k 8001/tcp")  # kills anything using port 8001


# run FastAPI in background
proc = subprocess.Popen(["uvicorn", "basic.put:app", "--reload", "--port=8001"])


INFO:     Will watch for changes in these directories: ['/home/ujjwal/learning/fastapi_learn']
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
INFO:     Started reloader process [130468] using WatchFiles
INFO:     Started server process [130470]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [30]:
!curl http://127.0.0.1:8001/

INFO:     127.0.0.1:39262 - "GET / HTTP/1.1" 200 OK
"Welcome to the home page"

In [31]:
## update the database using put 

!curl -X PUT "http://127.0.0.1:8001/reviews/" \
-H "Content-Type: application/json" \
-d '{"movie":"mahabarat","num_stars":5,"text":"oh my god","review_id":1}'


INFO:     127.0.0.1:39270 - "PUT /reviews/ HTTP/1.1" 200 OK
{"status":"updated","review":{"_id":1,"movie":"mahabarat","num_stars":5,"text":"oh my god"}}

In [32]:
!curl http://127.0.0.1:8001/reviews | python -m json.tool

INFO:     127.0.0.1:39286 - "GET /reviews HTTP/1.1" 200 OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   131  100   131    0     0   130k      0 --:--:-- --:--:-- --:--:--  127k
{
    "1": {
        "_id": 1,
        "movie": "mahabarat",
        "num_stars": 5,
        "text": "oh my god"
    },
    "2": {
        "_id": 2,
        "movie": "Matrix",
        "num_stars": 4,
        "text": "Awesome!"
    }
}


In [33]:
import os
os.system("fuser -k 8001/tcp")  # kills anything using port 8001


 130468 130470

8001/tcp:           


0